In [ ]:
import heapq
import pandas as pd
import folium
from IPython.display import display
from datetime import datetime

In [ ]:
def load_graph(file_path, user_input_time):
    df = pd.read_csv(file_path)
    graph = {}

    # 시간대에 따른 cost 선택
    if 7 <= user_input_time <= 9:
        cost_column = 'cost_morning'
    elif 18 <= user_input_time <= 20:
        cost_column = 'cost_evening'
    else:
        print("7시에서 9시 사이 혹은 18시에서 20시 사이의 시간대를 입력해주세요.")
        return None

    # 그래프 생성
    for _, row in df.iterrows():
        start, end, distance, cost = row['id_start'], row['id_end'], row['distance'], row[cost_column]

        if start not in graph:
            graph[start] = []
        if end not in graph:
            graph[end] = []

        # 유향 엣지로 추가
        graph[start].append((end, cost, distance))  # start -> end

        # 역방향 엣지 추가 여부 확인
        if not any(neighbor == start for neighbor, _, _ in graph[end]):
            graph[end].append((start, cost, distance))  # end -> start 

    return graph

In [ ]:
def heuristic(node, start, graph):
    # 반환점에서 출발지로의 비용을 직접 사용
    for neighbor, cost, _ in graph[start]:
        if neighbor == node:
            return cost
    return 0

In [ ]:
def a_star_search(graph, start, min_distance, max_distance):
    pq = []  # (f(n), g(n), 현재 노드, 경로, 거리)
    heapq.heappush(pq, (0, 0, start, [start], 0))
    best_cycle = None
    min_cost = float('inf')
    best_distance = 0  # 최적 경로의 거리 초기화

    exceeded_paths = set()
    best_path_str = ""
    visited_edges_map = {}

    print(f"Starting path exploration with A* from node {start}...\n")

    while pq:
        fn, gn, node, path, distance = heapq.heappop(pq)

        path_tuple = tuple(path)
        if path_tuple in exceeded_paths:
            continue

        if gn >= min_cost:
            print(f"Pruning path {path} with cost {gn} as it exceeds current min cost {min_cost}\n")
            continue

        if distance > max_distance:
            print(f"Exceeded max distance for path {path}: {distance}m\n")
            exceeded_paths.add(path_tuple)
            continue

        if node == start and len(path) > 1:
            if min_distance <= distance <= max_distance and gn < min_cost:
                best_cycle = path
                min_cost = gn
                best_distance = distance
                best_path_str = f"\nNew best cycle found: {path} with cost: {gn}, distance: {distance}m\n"
                print(best_path_str)
            continue

        if path_tuple not in visited_edges_map:
            visited_edges_map[path_tuple] = set((min(path[i], path[i + 1]), max(path[i], path[i + 1])) for i in range(len(path) - 1))
        visited_edges = visited_edges_map[path_tuple]

        for neighbor, edge_cost, edge_distance in graph[node]:
            edge = (min(node, neighbor), max(node, neighbor))
            if edge in visited_edges or (neighbor, node) in visited_edges:
                continue

            new_gn = gn + edge_cost
            new_distance = distance + edge_distance

            if new_gn >= min_cost:
                print(f"Pruning neighbor {neighbor} from path {path} with projected cost {new_gn}\n")
                continue

            try:
                hn = heuristic(neighbor, start, graph)
            except KeyError:
                hn = float('inf')

            fn = new_gn + hn
            heapq.heappush(pq, (fn, new_gn, neighbor, path + [neighbor], new_distance))

    print("\nPath exploration complete.\n")
    return best_cycle, min_cost, best_distance


In [ ]:
def plot_graph_folium(graph, marker_file, best_cycle):
    marker_df = pd.read_csv(marker_file, encoding='cp949')

    # 지도 초기화 (첫 번째 노드의 위치로 중심 설정)
    start_lat, start_lon = marker_df.iloc[0]['위도'], marker_df.iloc[0]['경도']
    m = folium.Map(location=[start_lat, start_lon], zoom_start=14)

    # 최적 경로 노드의 위치 설정 및 마커 추가
    positions = {}
    if best_cycle:
        for node_id in best_cycle:
            node_data = marker_df[marker_df['id'] == node_id].iloc[0]
            lat, lon = node_data['위도'], node_data['경도']
            positions[node_id] = (lat, lon)
            folium.Marker([lat, lon], popup=f'Node {node_id}', icon=folium.Icon(color='red')).add_to(m)

    # 최적 경로 엣지 그리기
    if best_cycle:
        for i in range(len(best_cycle) - 1):
            start_node = best_cycle[i]
            end_node = best_cycle[i + 1]
            if start_node in positions and end_node in positions:
                start_pos = positions[start_node]
                end_pos = positions[end_node]
                folium.PolyLine([start_pos, end_pos], color='blue', weight=2, opacity=0.5).add_to(m)

    # 지도 출력
    display(m)

In [ ]:
# 출발 노드 입력
user_input_node = float(input("출발노드: "))

# 거리 입력 (km 단위)
user_input_km = float(input("코스 거리: "))

# 시간대
user_input_time = float(input("시간대: "))

# 거리 제한 설정 (m 단위)
min_distance = int((user_input_km - 0.3) * 1000)
max_distance = int((user_input_km + 0.3) * 1000)

# 파일 경로 및 실행
file_path = "데이터_전처리_완료.csv"  # 실제 파일 경로로 교체
marker_file_path = "marker_id.csv"  # 노드 위치 정보가 담긴 파일 경로
graph = load_graph(file_path, user_input_time)
start_node = user_input_node

In [ ]:
# 최적 경로 탐색
try:
    cycle, cost, distance = a_star_search(graph, start_node, min_distance, max_distance)
    print("\n최적 사이클:", cycle)
    print("최소 비용:", cost)
    print("최적 경로의 총 거리:", distance)
except ValueError as e:
    print(f"Error during path search: {e}")

In [ ]:
# 그래프 시각화 (Folium 사용)
plot_graph_folium(graph, marker_file_path, cycle)